In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
!which python

/Users/davitbzhalava/opt/anaconda3/bin/python


In [7]:
from on_demand_feature import *
#from features import coord_to_postcode
#from on_demand_feature import coord_to_postcode

ModuleNotFoundError: No module named 'on_demand_feature'

In [27]:
df = pd.read_csv("../data/housing_zip.csv")

In [ ]:
df

In [ ]:
df.drop(['longitude', 'latitude'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df["ocean_proximity"].value_counts()

In [ ]:
df.describe()

In [29]:
df["pk"] = df.index

In [2]:
os.environ['HOPSWORKS_PROJECT']="ondemand_feature"
os.environ['HOPSWORKS_HOST']="8c7943b0-a2d3-11ed-8cee-11db34df42ef.cloud.hopsworks.ai/"
os.environ['HOPSWORKS_API_KEY']="rYKP15t4AHy0x9Xp.5EhDC1aatYdbm5JG4f8SCZzGTFXH254PVZ33BsiGjJ8KRU3vMifnGAU5idZWRLtP"

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c102de60-a072-11ed-9946-b5c95dfcd005.cloud.hopsworks.ai:443/p/1144
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
profile_fg = fs.get_or_create_feature_group(
    name="housing_fg",
    version=1,
    description="housing data",
    primary_key = ["pk"],
    online_enabled=True
)
profile_fg.insert(df)

In [ ]:
query = profile_fg.select_all()

In [ ]:
# Load the transformation functions.
label_encoder = fs.get_transformation_function(name="label_encoder")

# Map features to transformation functions.
transformation_functions = {
    "ocean_proximity": label_encoder,
}

In [ ]:
feature_view = fs.get_or_create_feature_view(
    name='housing_fv',
    version=1,
    query=query,
    transformation_functions=transformation_functions,
    labels=["median_house_value"],
)

In [ ]:
td_version, td_job = feature_view.create_train_test_split(
    description = 'housing_training_dataset_random_splitted',
    data_format = 'csv',
    test_size = 0.3,
    write_options = {'wait_for_job': False},
    coalesce = True,
)

In [7]:
feature_view = fs.get_feature_view(
    name='housing_fv',
    version=1)


In [8]:
train_x, test_x, train_y, test_y = feature_view.get_train_test_split(1)

In [9]:
train_x.drop("pk", axis=1, inplace=True)
test_x.drop("pk", axis=1, inplace=True)

In [10]:
train_x

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,zipcode,income_cat
0,1.0,83.0,15.0,32.0,15.0,4.8750,0,93505,4
1,1.0,2062.0,343.0,872.0,268.0,5.2636,0,0,4
2,1.0,2254.0,328.0,402.0,112.0,4.2500,0,95357,3
3,2.0,96.0,18.0,30.0,16.0,5.3374,0,0,4
4,2.0,158.0,43.0,94.0,57.0,2.5625,1,94608,2
...,...,...,...,...,...,...,...,...,...
14279,52.0,4513.0,985.0,1926.0,815.0,1.5923,0,96025,2
14280,52.0,4985.0,1355.0,1848.0,1255.0,4.9211,1,90214,4
14281,52.0,5486.0,1044.0,1977.0,754.0,2.1833,0,0,2
14282,52.0,5783.0,2747.0,4518.0,2538.0,1.7240,1,94108,2


In [11]:
import numpy as np
import xgboost as xgb

clf = xgb.XGBRegressor(verbosity=0)

clf.fit(train_x, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [12]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [13]:
import joblib
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

model_dir="hausing_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

pkl_file_name = model_dir + '/hausing_model.pkl'

joblib.dump(clf, pkl_file_name)

input_schema = Schema(train_x)
output_schema = Schema(train_y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)


model = mr.python.create_model(
    name="hausingmodel",
    description = "hausing Model",
    input_example = train_x.sample(),
    model_schema = model_schema
)

model.save(pkl_file_name)


  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c102de60-a072-11ed-9946-b5c95dfcd005.cloud.hopsworks.ai:443/p/1144/models/hausingmodel/1


Model(name: 'hausingmodel', version: 1)

In [15]:
%%writefile predict_example.py
import os
import numpy as np
import hsfs
import joblib
import geopy

def get_zip_code(x):
        geolocator = geopy.Nominatim(user_agent="check_1")
        location = geolocator.reverse("{}, {}".format(x['latitude'],x['longitude']))
        if 'address' in location.raw:
            if 'postcode' in location.raw['address']:        
                return int(location.raw['address']['postcode'])
            else:
                return 0
        else:
            return 0  

class Predict(object):      

    def __init__(self):
        """ Initializes the serving state, reads a trained model"""        
        # get feature store handle
        fs_conn = hsfs.connection()
        self.fs = fs_conn.get_feature_store()
        
        # get feature views
        self.fv = self.fs.get_feature_view("housing_fv", 1)
        
        # initialise serving
        self.fv.init_serving(1)

        # load the trained model
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/hausing_model.pkl")
        print("Initialization Complete")

    def predict(self, inputs):
        """ Serves a prediction request usign a trained model"""
        zipcode = get_zip_code({'longitude':inputs[1], 'latitude': inputs[2]})        
        feature_vector = self.fv.get_feature_vector({"pk": inputs[0]}, passed_features={"zipcode": zipcode})
        indexes_to_remove = [9]
        feature_vector = [i for j, i in enumerate(feature_vector) if j not in indexes_to_remove]
        
        return self.model.predict(np.asarray(feature_vector).reshape(1, -1)).tolist() # Numpy Arrays are not JSON serializable


Overwriting predict_example.py


In [16]:
import os
dataset_api = project.get_dataset_api()

uploaded_file_path = dataset_api.upload("predict_example.py", "Models", overwrite=True)
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)


Uploading: 0.000%|          | 0/1617 elapsed<00:00 remaining<?

In [18]:
mr = project.get_model_registry()
model = mr.get_model(name="hausingmodel",version=1)

try:
    ms = project.get_model_serving()
    deployment = ms.get_deployment("hausingmodeldeployment")
except:
    deployment = model.deploy(
        name="hausingmodeldeployment", 
        serving_tool="KSERVE",
        script_file=predictor_script_path
    )

Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
Deployment created, explore it at https://c102de60-a072-11ed-9946-b5c95dfcd005.cloud.hopsworks.ai:443/p/1144/deployments/6
Before making predictions, start the deployment by using `.start()`


In [20]:
state = deployment.get_state()

if state.status != "Running":
    deployment.start()
    deployment.describe()
else:
    print("Deployment already running")

Deployment is in failed state. Failed to be scheduled: 0/1 nodes are available: 1 Insufficient cpu.
{
    "artifact_version": 1,
    "batching_configuration": {
        "batching_enabled": false
    },
    "created": "2023-02-01T22:51:38.862Z",
    "creator": "Davit Bzhalava",
    "description": null,
    "id": 6,
    "inference_logging": "NONE",
    "model_framework": "PYTHON",
    "model_name": "hausingmodel",
    "model_path": "/Projects/ondemand_feature_2/Models/hausingmodel",
    "model_server": "PYTHON",
    "model_version": 1,
    "name": "hausingmodeldeployment",
    "predictor": "predict_example.py",
    "predictor_resources": {
        "limits": {
            "cores": 0.5,
            "gpus": 0,
            "memory": 1024
        },
        "requests": {
            "cores": 0.2,
            "gpus": 0,
            "memory": 32
        }
    },
    "requested_instances": 0,
    "serving_tool": "KSERVE"
}


In [23]:
deployment.get_state()

PredictorState(status: 'Idle')

[0.0, -122.23, 37.88]

In [ ]:
for inputs in df[["pk", "longitude", "latitude"]].values:
    data = {
        "inputs": inputs.tolist()
    }

    deployment.predict(data)